> Ce qui suit est une adaptation de : 

https://towardsdatascience.com/implementing-a-fully-convolutional-network-fcn-in-tensorflow-2-3c46fb61de3b

Avec :

https://github.com/himanshurawlani/fully_convolutional_network

# Création d'un FCN

In [2]:
import tensorflow as tf
from tensorflow import keras

In [3]:
# tf.config.list_physical_devices("GPU")

In [4]:
def FCN_model(len_classes=5, dropout_rate=0.2):
    
    input = tf.keras.layers.Input(shape=(None, None, 3)) # Les tailles None permettent de prendre n'importe quelle taille d'image en entrée

    x = tf.keras.layers.Conv2D(filters=32, kernel_size=3, strides=1)(input)
    #x = tf.keras.layers.Dropout(dropout_rate)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation('relu')(x)

    # x = tf.keras.layers.MaxPooling2D()(x)

    x = tf.keras.layers.Conv2D(filters=64, kernel_size=3, strides=1)(x)
    #x = tf.keras.layers.Dropout(dropout_rate)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation('relu')(x)

    # x = tf.keras.layers.MaxPooling2D()(x)

    x = tf.keras.layers.Conv2D(filters=128, kernel_size=3, strides=2)(x)
    #x = tf.keras.layers.Dropout(dropout_rate)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation('relu')(x)

    # x = tf.keras.layers.MaxPooling2D()(x)

    x = tf.keras.layers.Conv2D(filters=256, kernel_size=3, strides=2)(x)
    #x = tf.keras.layers.Dropout(dropout_rate)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation('relu')(x)

    # x = tf.keras.layers.MaxPooling2D()(x)

    x = tf.keras.layers.Conv2D(filters=512, kernel_size=3, strides=2)(x)
    #x = tf.keras.layers.Dropout(dropout_rate)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation('relu')(x)

    # # Fully connected layer 1
    x = tf.keras.layers.Conv2D(filters=64, kernel_size=1, strides=1)(x)
    #x = tf.keras.layers.Dropout(dropout_rate)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation('relu')(x)

    # # Fully connected layer 2
    x = tf.keras.layers.Conv2D(filters=len_classes, kernel_size=1, strides=1)(x)
    #x = tf.keras.layers.Dropout(dropout_rate)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.GlobalMaxPooling2D()(x)
    predictions = tf.keras.layers.Activation('softmax')(x)

    model = tf.keras.Model(inputs=input, outputs=predictions)
    
    print(model.summary())
    print(f'Total number of layers: {len(model.layers)}')

    return model


In [5]:
FCN_model(len_classes=162, dropout_rate=0.2) # 162 classes car 162 champions différents

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, None, None, 3)]   0         
_________________________________________________________________
conv2d (Conv2D)              (None, None, None, 32)    896       
_________________________________________________________________
batch_normalization (BatchNo (None, None, None, 32)    128       
_________________________________________________________________
activation (Activation)      (None, None, None, 32)    0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, None, None, 64)    18496     
_________________________________________________________________
batch_normalization_1 (Batch (None, None, None, 64)    256       
______________________________________________

### Création des images train et val 

In [6]:
import os, cv2
from shutil import copy2
import numpy as np
import tensorflow as tf 
import random


def crop_blur(xmin,xmax,ymin,ymax, resize, blur, bright, image):
    """
    Crop and blur an image

    Args
    ----
    xmin: int
        x à partir duquel on coupe l'image
    xmax: int
        x jusqu'auquel on coupe l'image (exemple : -80)
    ymin: int
        y à partir duquel on coupe l'image
    ymax: int
        y jusqu'auquel on coupe l'image (exemple : -90)
    resize: float
        facteur de redimensionnement de l'image
    blur: int
        taille du filtre de flou gaussien
    image: np.array
    """
    image = image[ymin:ymax, xmin:xmax]
    image = cv2.resize(image, (0, 0), fx=resize, fy=resize)
    image = cv2.GaussianBlur(image, (blur,blur), 0)
    # Pour finir, on fait varier la luminosité de l'image
    image = cv2.addWeighted(image, bright, np.zeros(image.shape, image.dtype), 0, 0)
    return image

def cercle_random(image, N=2.5):
    x_pos = random.randint(int(image.shape[1]/N), int(image.shape[1]) - int(image.shape[1]/N) - 1)
    y_pos = random.randint(int(image.shape[0]/N), int(image.shape[0]) - int(image.shape[0]/N) - 1)

    centre_cercle = (y_pos, x_pos)
    rayon_cercle = random.randint(min(int(image.shape[0]/N),int(image.shape[0]/N))-1, min(x_pos,y_pos,int(image.shape[0]-y_pos),int(image.shape[1]-x_pos)))

    mask = np.zeros_like(image)
    mask = cv2.circle(mask, centre_cercle, rayon_cercle, (255,255,255), -1)
    image = cv2.bitwise_and(image, mask)
    return(image)

def rectangle_random(image):
    mask = np.zeros(image.shape,np.uint8)
    x_min = random.randint(0, 4)
    y_min = random.randint(0, 4)
    x_max = random.randint(image.shape[0]-4, image.shape[0]-1)
    y_max = random.randint(image.shape[1]-4, image.shape[1]-1)
    mask[y_min:y_max,x_min:x_max] = image[y_min:y_max,x_min:x_max]
    return(mask)


def split_dataset(PATH_DOSS = './images_draft', DATASET_PATH = './dataset', train_images = 750, val_images = 60):
    # Specify path to the folder with all the images
    nom_champions = os.listdir(PATH_DOSS)
    # # on enlève la fin des noms des fichiers en "_0.jpg" pour avoir juste le nom du champion
    # for i in range(len(nom_champions)):
    #     nom_champions[i] = nom_champions[i][:-6]


    # On récupère 45 images pour chaque champion pour l'entrainement, et 12 pour la validation
    noms_champions_train = []
    noms_champions_val = []
    for champion in nom_champions:
        for i in range(90):
            noms_champions_train.append(champion+f"{i}")
            if i<22:
                noms_champions_val.append(champion+f"{i}")

    # On en récupère "train_images" pour l'entrainement et "val_images" pour la validation que l'on met dans le dossier dataset au bon endroit
    train_champions = noms_champions_train.copy()
    val_champions = noms_champions_val.copy()
    # Specify path for copying the dataset into train and val sets
    os.makedirs(DATASET_PATH, exist_ok=True)

    # Creating train directory
    train_dir = os.path.join(DATASET_PATH, 'train')
    os.makedirs(train_dir, exist_ok=True)

    # Creating val directory
    val_dir = os.path.join(DATASET_PATH, 'val')
    os.makedirs(val_dir, exist_ok=True) 

    #-------------------------------------------------#
    # On crée des paramètres random pour chaque image
    param_possibles = [(x,y,z,a,b,c,bright) for x in [50] for y in [-80,-90,-100] for z in [60] for a in [-100,-85] for b in [0.15,0.20,0.25,0.30] for c in [3,5,7] for bright in [0.7,0.8,0.9,1]]
    param_possibles_bans = [(x,-200+x,x,-200+x,resize,blur,bright) for x in range(70,100,10) for resize in [0.16,0.18,0.20,0.22,0.24] for blur in [3] for bright in [0.8,0.85,0.9,0.95,1]]
    param_possibles_picks = [(x1,-160+x1,y1,-160+y1,resize,blur,bright) for x1 in range(25,125,25) for y1 in range(25,100,25) for resize in [0.20,0.24,0.27] for blur in [3,5] for bright in [0.7,0.8,0.9,1]]

    champions = ['Aatrox', 'Ahri', 'Akali', 'Akshan', 'Alistar', 'Amumu', 'Anivia', 'Annie', 'Aphelios', 'Ashe', 'AurelionSol', 'Azir', 'Bard', 'Belveth', 'Blitzcrank', 'Brand', 'Braum', 'Caitlyn', 'Camille', 'Cassiopeia', 'Chogath', 'Corki', 'Darius', 'Diana', 'Draven', 'DrMundo', 'Ekko', 'Elise', 'Evelynn', 'Ezreal', 'FiddleSticks', 'Fiora', 'Fizz', 'Galio', 'Gangplank', 'Garen', 'Gnar', 'Gragas', 'Graves', 'Gwen', 'Hecarim', 'Heimerdinger', 'Illaoi', 'Irelia', 'Ivern', 'Janna', 'JarvanIV', 'Jax', 'Jayce', 'Jhin', 'Jinx', 'Kaisa', 'Kalista', 'Karma', 'Karthus', 'Kassadin', 'Katarina', 'Kayle', 'Kayn', 'Kennen', 'Khazix', 'Kindred', 'Kled', 'KogMaw', 'KSante', 'Leblanc', 'LeeSin', 'Leona', 'Lillia', 'Lissandra', 'Lucian', 'Lulu', 'Lux', 'Malphite', 'Malzahar', 'Maokai', 'MasterYi', 'MissFortune', 'MonkeyKing', 'Mordekaiser', 'Morgana', 'Nami', 'Nasus', 'Nautilus', 'Neeko', 'Nidalee', 'Nilah', 'Nocturne', 'Nunu', 'Olaf', 'Orianna', 'Ornn', 'Pantheon', 'Poppy', 'Pyke', 'Qiyana', 'Quinn', 'Rakan', 'Rammus', 'RekSai', 'Rell', 'Renata', 'Renekton', 'Rengar', 'Riven', 'Rumble', 'Ryze', 'Samira', 'Sejuani', 'Senna', 'Seraphine', 'Sett', 'Shaco', 'Shen', 'Shyvana', 'Singed', 'Sion', 'Sivir', 'Skarner', 'Sona', 'Soraka', 'Swain', 'Sylas', 'Syndra', 'TahmKench', 'Taliyah', 'Talon', 'Taric', 'Teemo', 'Thresh', 'Tristana', 'Trundle', 'Tryndamere', 'TwistedFate', 'Twitch', 'Udyr', 'Urgot', 'Varus', 'Vayne', 'Veigar', 'Velkoz', 'Vex', 'Vi', 'Viego', 'Viktor', 'Vladimir', 'Volibear', 'Warwick', 'Xayah', 'Xerath', 'XinZhao', 'Yasuo', 'Yone', 'Yorick', 'Yuumi', 'Zac', 'Zed', 'Zeri', 'Ziggs', 'Zilean', 'Zoe', 'Zyra']
    param_par_champ_train = {}
    param_par_champ_val = {}
    for champ in champions:
        param_par_champ_train[champ] = {}
        param_par_champ_val[champ] = {}
        # On choisit aux hasard un certain nombre de configurations
        choix = random.choices(param_possibles, k=10)
        param_par_champ_train[champ]["quality"] = choix
        poss_val = [tuple for tuple in param_possibles if tuple not in choix]
        param_par_champ_val[champ]["quality"] = random.choices(poss_val, k=4)

        choix = random.choices(param_possibles_bans, k=40)
        param_par_champ_train[champ]["ban"] = choix
        poss_val = [tuple for tuple in param_possibles_bans if tuple not in choix]
        param_par_champ_val[champ]["ban"] = random.choices(poss_val, k=9)

        choix = random.choices(param_possibles_picks, k=40)
        param_par_champ_train[champ]["pick"] = choix
        poss_val = [tuple for tuple in param_possibles_picks if tuple not in choix]
        param_par_champ_val[champ]["pick"] = random.choices(poss_val, k=9)


    #-------------------------------------------------#
    # print(train_champions)

    j=0 # compteur pour les images de chaque type
    for i in range(len(train_champions)):
        # Images de qualité réduite
        if j < 10 :
            if j < 10 :
                nom_image = train_champions[i][:-1]
                deb = -1
            else :
                nom_image = train_champions[i][:-2]
                deb = -2
            image = cv2.imread(f"{PATH_DOSS}/{nom_image}/{nom_image}_0.jpg")
            tuple = param_par_champ_train[nom_image]["quality"][j]
            image_pick = crop_blur(tuple[0],tuple[1],tuple[2],tuple[3],tuple[4],tuple[5],tuple[6],image)
            cv2.imwrite(f"{DATASET_PATH}/train/{nom_image}_{train_champions[i][deb:]}.jpg", image_pick)
            j+=1


        # Images de qualité réduite, avec des ronds (simili pick)
        elif j < 50 :
            nom_image = train_champions[i][:-2]
            image = cv2.imread(f"{PATH_DOSS}/{nom_image}/{nom_image}_0.jpg")
            tuple = param_par_champ_train[nom_image]["pick"][j-10]
            image_pick = crop_blur(tuple[0],tuple[1],tuple[2],tuple[3],tuple[4],tuple[5],tuple[6],image)
            image_pick = cercle_random(image_pick)
            cv2.imwrite(f"dataset/train/{nom_image}_{train_champions[i][-2:]}.jpg", image_pick)
            j+=1

        # Images de qualité réduite, avec des rectangles (simili ban)
        else :
            nom_image = train_champions[i][:-2]
            image = cv2.imread(f"{PATH_DOSS}/{nom_image}/{nom_image}_0.jpg")
            tuple = param_par_champ_train[nom_image]["ban"][j-50]
            image_ban = crop_blur(tuple[0],tuple[1],tuple[2],tuple[3],tuple[4],tuple[5],tuple[6],image)
            mask = rectangle_random(image_ban)
            cv2.imwrite(f"dataset/train/{nom_image}_{train_champions[i][-2:]}.jpg", mask)
            j+=1
            if j == 90 : # On a bouclé sur toutes les images d'un champion
                j = 0

    j=0 # compteur pour les images de chaque type pour la validation
    for i in range(len(val_champions)):

        if j < 4 :
            nom_image = val_champions[i][:-1]
            image = cv2.imread(f"{PATH_DOSS}/{nom_image}/{nom_image}_0.jpg")
            tuple = param_par_champ_val[nom_image]["quality"][j]
            image_pick = crop_blur(tuple[0],tuple[1],tuple[2],tuple[3],tuple[4],tuple[5],tuple[6],image)
            cv2.imwrite(f"{DATASET_PATH}/val/{nom_image}_{val_champions[i][-1:]}.jpg", image_pick)
            j+=1

        # Images de qualité réduite, avec des ronds (simili pick)
        elif j < 13 :
            if j < 10 : 
                nom_image = val_champions[i][:-1]
                deb = -1
            else : 
                nom_image = val_champions[i][:-2]
                deb = -2
            image = cv2.imread(f"{PATH_DOSS}/{nom_image}/{nom_image}_0.jpg")
            tuple = param_par_champ_val[nom_image]["pick"][j-4]
            image_pick = crop_blur(tuple[0],tuple[1],tuple[2],tuple[3],tuple[4],tuple[5],tuple[6],image)
            image_pick = cercle_random(image_pick)
            cv2.imwrite(f"dataset/val/{nom_image}_{val_champions[i][deb:]}.jpg", image_pick)
            j+=1

        # Images de qualité réduite, avec des rectangles (simili ban)
        else :
            nom_image = val_champions[i][:-2]
            image = cv2.imread(f"{PATH_DOSS}/{nom_image}/{nom_image}_0.jpg")
            tuple = param_par_champ_val[nom_image]["ban"][j-13]
            image_ban = crop_blur(tuple[0],tuple[1],tuple[2],tuple[3],tuple[4],tuple[5],tuple[6],image)
            mask = rectangle_random(image_ban)
            cv2.imwrite(f"dataset/val/{nom_image}_{val_champions[i][-2:]}.jpg", mask)
            j+=1
            if j == 22 : # On a bouclé sur toutes les images d'un champion
                j = 0

In [7]:
split_dataset()

### Création de ce qui permet de faire le dataset avec les bons labels et les bonnes tailles

In [8]:
import os
import numpy as np
import cv2
from sklearn import preprocessing
import tensorflow as tf


class Generator(tf.keras.utils.Sequence):

    def __init__(self, DATASET_PATH, BATCH_SIZE=32, shuffle_images=True, image_min_side=24):
        """ Initialize Generator object.
        Args
            DATASET_PATH           : Path to folder containing individual folders named by their class names
            BATCH_SIZE             : The size of the batches to generate.
            shuffle_images         : If True, shuffles the images read from the DATASET_PATH
            image_min_side         : After resizing the minimum side of an image is equal to image_min_side.
        """

        self.batch_size = BATCH_SIZE
        self.shuffle_images = shuffle_images
        self.image_min_side = image_min_side
        self.load_image_paths_labels(DATASET_PATH)
        self.create_image_groups()
    
    def load_image_paths_labels(self, DATASET_PATH):
        nom_champions_gen = os.listdir('./images_draft')
        # Le label binarizer permet de transformer les labels en vecteurs de 0 et de 1
        lb = preprocessing.LabelBinarizer()
        lb.fit(nom_champions_gen)
        # On récupère le nombre de classes, on en a bien 162    
        # print(lb.classes_)

# On lit chaque image et on la met dans une liste, on rajoute avec son label en prenant garde de supprimer le numéro de l'image
        self.image_paths = []
        self.image_labels = []
        for champ in os.listdir(DATASET_PATH):
            champ_nom_temp = os.path.join(DATASET_PATH, champ)
            self.image_paths.append(os.path.join(DATASET_PATH, champ))
            self.image_labels.append(champ[:champ.rfind('_')])
        

        self.image_labels = np.array(lb.transform(self.image_labels), dtype='float32')
        print(self.image_labels.shape)
        
        assert len(self.image_paths) == len(self.image_labels)

    def create_image_groups(self):
        if self.shuffle_images:
            # Randomly shuffle dataset
            seed = 4321
            np.random.seed(seed)
            np.random.shuffle(self.image_paths)
            np.random.seed(seed)
            np.random.shuffle(self.image_labels)

        # Divide image_paths and image_labels into groups of BATCH_SIZE
        self.image_groups = [[self.image_paths[x % len(self.image_paths)] for x in range(i, i + self.batch_size)]
                              for i in range(0, len(self.image_paths), self.batch_size)]
        self.label_groups = [[self.image_labels[x % len(self.image_labels)] for x in range(i, i + self.batch_size)]
                              for i in range(0, len(self.image_labels), self.batch_size)]

    def resize_image(self, img, min_side_len):

        h, w, c = img.shape

        # limit the min side maintaining the aspect ratio
        if min(h, w) < min_side_len:
            im_scale = float(min_side_len) / h if h < w else float(min_side_len) / w
        else:
            im_scale = 1.

        new_h = int(h * im_scale)
        new_w = int(w * im_scale)

        re_im = cv2.resize(img, (new_w, new_h), interpolation=cv2.INTER_AREA)
        return re_im, new_h / h, new_w / w

    def load_images(self, image_group):

        images = []
        for image_path in image_group:
            img = cv2.imread(image_path)
            img_shape = len(img.shape)
            if img_shape == 2:
                img = cv2.cvtColor(img,cv2.COLOR_GRAY2RGB)
            elif img_shape == 4:
                img = cv2.cvtColor(img,cv2.COLOR_BGRA2RGB)
            elif img_shape == 3:
                img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img, rh, rw = self.resize_image(img, self.image_min_side)
            images.append(img)

        return images

    def construct_image_batch(self, image_group):
        # get the max image shape
        max_shape = tuple(max(image.shape[x] for image in image_group) for x in range(3))

        # construct an image batch object
        image_batch = np.zeros((self.batch_size,) + max_shape, dtype='float32')

        # copy all images to the upper left part of the image batch object
        for image_index, image in enumerate(image_group):
            image_batch[image_index, :image.shape[0], :image.shape[1], :image.shape[2]] = image

        return image_batch
    
    def __len__(self):
        """
        Number of batches for generator.
        """

        return len(self.image_groups)

    def __getitem__(self, index):
        """
        Keras sequence method for generating batches.
        """
        image_group = self.image_groups[index]
        label_group = self.label_groups[index]
        images = self.load_images(image_group)
        image_batch = self.construct_image_batch(images)

        return np.array(image_batch), np.array(label_group)



In [9]:
BASE_PATH = 'dataset'
train_generator = Generator('dataset/train')
val_generator = Generator('dataset/val')
print(len(train_generator))
print(len(val_generator))
image_batch, label_group = train_generator.__getitem__(0)
print(image_batch.shape)
print(label_group.shape)

(14580, 162)
(3564, 162)
456
112
(32, 59, 59, 3)
(32, 162)


### train du modèle

Pour fixer le bug du KeyError : 'accuracy' : remplacer accuracy par acc dans la cellule ci-dessous (dans metrics = ... et dans model_path = ...)

In [10]:
import tensorflow as tf
import os

def train(model, train_generator, val_generator, nom_model, epochs = 50):
    model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.0001),
                    loss='categorical_crossentropy',
                    metrics=['acc'])

    checkpoint_path = './snapshots'
    os.makedirs(checkpoint_path, exist_ok=True)
    model_path = os.path.join(checkpoint_path, 'model_epoch_{epoch:02d}_loss_{loss:.2f}_acc_{acc:.2f}_val_loss_{val_loss:.2f}_val_acc_{val_acc:.2f}.h5')
    
    history = model.fit_generator(generator=train_generator,
                                    steps_per_epoch=len(train_generator),
                                    epochs=epochs,
                                    callbacks=[tf.keras.callbacks.ModelCheckpoint(model_path, monitor='val_loss', save_best_only=True, verbose=1)],
                                    validation_data=val_generator,
                                    validation_steps=len(val_generator))

    model.save(f"recognition_model")

    return history
    

In [11]:
# Create FCN model
model = FCN_model(len_classes=162, dropout_rate=0.2)

# The below folders are created using utils.py
train_dir = 'dataset/train'
val_dir = 'dataset/val'

# If you get out of memory error try reducing the batch size
BATCH_SIZE=5
train_generator = Generator(train_dir, BATCH_SIZE, shuffle_images=True, image_min_side=24)
val_generator = Generator(val_dir, BATCH_SIZE, shuffle_images=True, image_min_side=24)

EPOCHS=7
history = train(model, train_generator, val_generator, "saved_model",  epochs=EPOCHS)

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, None, None, 3)]   0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, None, None, 32)    896       
_________________________________________________________________
batch_normalization_7 (Batch (None, None, None, 32)    128       
_________________________________________________________________
activation_7 (Activation)    (None, None, None, 32)    0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, None, None, 64)    18496     
_________________________________________________________________
batch_normalization_8 (Batch (None, None, None, 64)    256       
_________________________________________________________________
activation_8 (Activation)    (None, None, None, 64)    0   

2023-02-19 21:20:58.193476: I tensorflow/stream_executor/platform/default/dso_loader.cc:97] Successfully opened dynamic library libdirectml.d6f03b303ac3c4f2eeb8ca631688c9757b361310.so
2023-02-19 21:20:58.193615: I tensorflow/stream_executor/platform/default/dso_loader.cc:97] Successfully opened dynamic library libdxcore.so
2023-02-19 21:20:58.196983: I tensorflow/stream_executor/platform/default/dso_loader.cc:97] Successfully opened dynamic library libd3d12.so
2023-02-19 21:20:58.401998: I tensorflow/core/common_runtime/dml/dml_device_cache.cc:250] DirectML device enumeration: found 1 compatible adapters.
2023-02-19 21:20:58.404788: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2023-02-19 21:20:58.405276: I tensorflow/core/common_runtime/dml/dml_device_cache.cc:186] DirectML: creating device on adapter 0 (Radeon RX 580 Series)


Epoch 1/7
 713/2916 [======>.......................] - ETA: 20s - loss: 1.6457 - acc: 0.6957
Epoch 00001: val_loss improved from inf to 1.64565, saving model to ./snapshots/model_epoch_01_loss_3.55_acc_0.32_val_loss_1.65_val_acc_0.70.h5
2916/2916 [==============================] - 99s 34ms/step - loss: 3.5512 - acc: 0.3208 - val_loss: 1.6457 - val_acc: 0.6957
Epoch 2/7
 713/2916 [======>.......................] - ETA: 20s - loss: 0.7017 - acc: 0.8670
Epoch 00002: val_loss improved from 1.64565 to 0.70170, saving model to ./snapshots/model_epoch_02_loss_1.59_acc_0.76_val_loss_0.70_val_acc_0.87.h5
2916/2916 [==============================] - 97s 33ms/step - loss: 1.5896 - acc: 0.7640 - val_loss: 0.7017 - val_acc: 0.8670
Epoch 3/7
 711/2916 [======>.......................] - ETA: 20s - loss: 0.4965 - acc: 0.9063
Epoch 00003: val_loss improved from 0.70170 to 0.49653, saving model to ./snapshots/model_epoch_03_loss_0.85_acc_0.91_val_loss_0.50_val_acc_0.91.h5
2916/2916 [====================